In [ ]:
import arxiv
import openai
import requests
import base64
import time
import hashlib
import json
from datetime import datetime, timedelta, timezone
from IPython.display import display, Markdown, Image
from requests.exceptions import RequestException
from io import BytesIO

# Load configuration and secrets
from config import get_config

# Get all configuration
config = get_config()

# Initialize the OpenAI client with API key from config
client = openai.OpenAI(api_key=config['OPENAI_API_KEY'])

# Extract config values for easier use
THREADS_USER_ID = config['THREADS_USER_ID']
THREADS_ACCESS_TOKEN = config['THREADS_ACCESS_TOKEN']
GITHUB_TOKEN = config['GITHUB_TOKEN']
GITHUB_REPO = config['GITHUB_REPO']
GITHUB_PAGES_SITE = config['GITHUB_PAGES_SITE']

def generate_harvard_reference(paper):
    authors = paper.authors
    
    if len(authors) == 1:
        author_str = authors[0].name
    elif len(authors) == 2:
        author_str = f"{authors[0].name} and {authors[1].name}"
    else:
        author_str = f"{authors[0].name} et al."
    
    year = paper.published.year
    title = paper.title
    
    reference = f"{author_str} ({year}) '{title}', arXiv preprint arXiv:{paper.get_short_id()}."
    
    return reference

def fetch_latest_papers(categories, max_results=12):  # Fixed: was max_results=1
    client_arxiv = arxiv.Client()  # Renamed to avoid confusion with OpenAI client
    last_week = datetime.now(timezone.utc) - timedelta(days=7)
    
    category_query = " OR ".join([f"cat:{cat}" for cat in categories])
    
    search = arxiv.Search(
        query = f"({category_query})",
        max_results = max_results,
        sort_by = arxiv.SortCriterion.SubmittedDate
    )
    
    results = list(client_arxiv.results(search))
    return [paper for paper in results if paper.published.replace(tzinfo=timezone.utc) > last_week]

def generate_blog_post(paper):
    authors = ', '.join([author.name for author in paper.authors])
    prompt = f"""Write an engaging blog post about the following scientific paper:

Title: {paper.title}
Authors: {authors}
Abstract: {paper.summary}

The blog post should:
1. Explain the main findings in simple terms
2. Discuss potential real-world implications
3. Be engaging and accessible to a general audience
4. Be around 300-400 words long

Blog Post:"""

    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that writes engaging blog posts about scientific papers."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=500,
            temperature=0.7
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error generating blog post: {e}")
        return None

def generate_threads_post(paper, blog_post_url):
    prompt = f"""Create a short, engaging post for Threads (max 350 characters) about this scientific paper:
    Title: {paper.title}
    
    Include a brief highlight of the research and its potential impact. 
    Do not include any hashtags or 'Read more' statements.
    """
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that creates engaging social media posts about scientific papers."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=100,
            temperature=0.7
        )
        threads_text = response.choices[0].message.content.strip().replace(":", "")
        
        custom_hashtags = "#AI #ArtificialIntelligence #MachineLearning #DataScience #Latest #Research #Arxiv #OpenAI"
        
        full_post = f"{threads_text}\n\n{custom_hashtags}\n\nRead more: {blog_post_url}"
        
        if len(full_post) > 500:
            available_chars = 500 - len(custom_hashtags) - len(blog_post_url) - 15
            truncated_text = threads_text[:available_chars-3] + "..."
            full_post = f"{truncated_text}\n\n{custom_hashtags}\n\nRead more: {blog_post_url}"
        
        return full_post
    except Exception as e:
        print(f"Error generating Threads post: {e}")
        return None
   
def generate_ai_image(paper, threads_post):
    """
    Generate an AI image using DALL-E 3 based on the research paper
    """
    # Create a more specific prompt based on the paper's content
    # Extract key concepts from the paper title and abstract
    title_words = paper.title.lower()
    
    # Determine the research domain for better image prompts
    if any(word in title_words for word in ['neural', 'network', 'deep', 'learning', 'ai', 'artificial']):
        domain = "neural networks and AI"
        visual_style = "futuristic digital networks with glowing nodes and connections"
    elif any(word in title_words for word in ['computer', 'vision', 'image', 'visual']):
        domain = "computer vision"
        visual_style = "digital image processing with geometric patterns and visual data"
    elif any(word in title_words for word in ['nlp', 'language', 'text', 'linguistic']):
        domain = "natural language processing"
        visual_style = "flowing text and language symbols transforming into digital patterns"
    elif any(word in title_words for word in ['robot', 'autonomous', 'control']):
        domain = "robotics"
        visual_style = "sleek robotic elements and autonomous systems"
    elif any(word in title_words for word in ['data', 'analysis', 'mining']):
        domain = "data science"
        visual_style = "abstract data visualizations and flowing information streams"
    else:
        domain = "artificial intelligence research"
        visual_style = "abstract technological concepts with clean, modern design"
    
    # Create a sophisticated prompt
    prompt = f"""
    Create a modern, professional illustration representing {domain} research. 
    The image should feature {visual_style}, using a clean and sophisticated 
    color palette with blues, purples, and subtle gradients. The style should be 
    minimalist yet engaging, suitable for a technology blog. Avoid text, people, 
    or specific logos. Focus on abstract technological concepts that convey 
    innovation and scientific progress.
    """
    
    try:
        print(f"Generating DALL-E 3 image for: {paper.title[:50]}...")
        
        response = client.images.generate(
            model="dall-e-3",  # Upgraded to DALL-E 3
            prompt=prompt,
            n=1,
            size="1024x1024",
            quality="standard",  # Use "hd" for higher quality but slower generation
            style="natural"     # "natural" or "vivid" - natural is more realistic
        )
        
        image_url = response.data[0].url
        print("✅ DALL-E 3 image generated successfully!")
        
        # Display the image in the notebook (if running in Jupyter)
        try:
            from IPython.display import display, Image
            display(Image(url=image_url))
        except ImportError:
            print(f"Image URL: {image_url}")
        
        return image_url
        
    except Exception as e:
        print(f"❌ Error generating DALL-E 3 image: {e}")
        
        # Fallback: try with a simpler prompt
        try:
            print("Trying with simplified prompt...")
            simple_prompt = f"A clean, modern illustration representing {domain}, minimalist style, blue and purple gradient background"
            
            response = client.images.generate(
                model="dall-e-3",
                prompt=simple_prompt,
                n=1,
                size="1024x1024",
                quality="standard"
            )
            
            image_url = response.data[0].url
            print("✅ Fallback image generated successfully!")
            return image_url
            
        except Exception as e2:
            print(f"❌ Fallback also failed: {e2}")
            return None

def download_and_save_image(image_url, paper_short_id, date):
    """
    Download the AI image and save it to GitHub repository
    This prevents the image from expiring
    """
    import requests
    import base64
    
    try:
        # Download the image
        print("Downloading image...")
        img_response = requests.get(image_url)
        img_response.raise_for_status()
        
        # Create filename
        image_filename = f"assets/images/{date}-{paper_short_id}.png"
        
        # Encode image for GitHub API
        encoded_image = base64.b64encode(img_response.content).decode("utf-8")
        
        # Upload to GitHub
        url = f"https://api.github.com/repos/{GITHUB_REPO}/contents/{image_filename}"
        headers = {
            "Authorization": f"token {GITHUB_TOKEN}",
            "Accept": "application/vnd.github.v3+json"
        }
        
        data = {
            "message": f"Add image for blog post {paper_short_id}",
            "content": encoded_image
        }
        
        response = requests.put(url, headers=headers, json=data)
        
        if response.status_code == 201:
            # Return the GitHub-hosted image URL
            github_image_url = f"https://{GITHUB_PAGES_SITE}/{image_filename}"
            print(f"✅ Image saved to GitHub: {github_image_url}")
            return github_image_url
        else:
            print(f"❌ Failed to save image to GitHub: {response.status_code}")
            print(f"Response: {response.text}")
            return image_url  # Return original URL as fallback
            
    except Exception as e:
        print(f"❌ Error saving image: {e}")
        return image_url  # Return original URL as fallback

def create_github_blog_post(paper, content, date, short_id, image_url):
    # Use consistent short_id for the file name
    file_name = f"{date}-{short_id}.md"
    
    # Generate Harvard reference
    harvard_reference = generate_harvard_reference(paper)
    
    # Fixed: No indentation in front matter
    file_content = f"""---
layout: post
title: "{paper.title}"
date: {date} {datetime.now().strftime('%H:%M:%S +0000')}
categories: [blog, AI, research]
image: {image_url}
---
![AI Generated Image]({image_url})

{content}

## Original Research Paper
For more details, please refer to the original research paper:
[{paper.title}]({paper.entry_id})

## Reference
{harvard_reference}
"""
    
    encoded_content = base64.b64encode(file_content.encode("utf-8")).decode("utf-8")
    url = f"https://api.github.com/repos/{GITHUB_REPO}/contents/_posts/{file_name}"
    headers = {
        "Authorization": f"token {GITHUB_TOKEN}",
        "Accept": "application/vnd.github.v3+json"
    }
    
    # Check if file already exists
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        print(f"Blog post already exists: {file_name}")
        return False, ""

    # File doesn't exist, create new file
    data = {
        "message": f"Add new blog post: {paper.title}",
        "content": encoded_content
    }

    response = requests.put(url, headers=headers, json=data)
    if response.status_code != 201:
        print(f"GitHub API Error: {response.status_code}")
        print(f"Response content: {response.text}")
        return False, ""
    
    # Construct the URL based on the file name
    post_url = f"https://{GITHUB_PAGES_SITE}/{date.replace('-', '/')}/{short_id}/"
    return True, post_url
   

def check_existing_post(short_id, date):
    file_name = f"{date}-{short_id}.md"
    url = f"https://api.github.com/repos/{GITHUB_REPO}/contents/_posts/{file_name}"
    headers = {
        "Authorization": f"token {GITHUB_TOKEN}",
        "Accept": "application/vnd.github.v3+json"
    }
    response = requests.get(url, headers=headers)
    return response.status_code == 200

def create_media_container(access_token, user_id, text, image_url):
    url = f"https://graph.threads.net/v1.0/{user_id}/threads"
    
    params = {
        "media_type": "IMAGE",
        "image_url": image_url,
        "text": text,
        "access_token": access_token
    }
    
    try:
        response = requests.post(url, params=params)
        response.raise_for_status()
        print(f"Create Media Container Status Code: {response.status_code}")
        return response.json()
    except requests.RequestException as e:
        print(f"Error creating media container: {e}")
        if hasattr(e, 'response') and e.response is not None:
            print(f"Response content: {e.response.text}")
        return None

def publish_thread(access_token, user_id, creation_id):
    url = f"https://graph.threads.net/v1.0/{user_id}/threads_publish"
    
    params = {
        "creation_id": creation_id,
        "access_token": access_token
    }
    
    try:
        response = requests.post(url, params=params)
        response.raise_for_status()
        print(f"Publish Thread Status Code: {response.status_code}")
        return response.json()
    except requests.RequestException as e:
        print(f"Error publishing thread: {e}")
        if hasattr(e, 'response') and e.response is not None:
            print(f"Response content: {e.response.text}")
        return None

def post_to_threads(text, image_url, access_token, user_id, initial_wait=30, max_retries=3):
    for attempt in range(max_retries):
        try:
            # Step 1: Create media container
            container = create_media_container(access_token, user_id, text, image_url)
            if container is None or 'id' not in container:
                print("Failed to create media container.")
                return False

            container_id = container['id']
            print(f"Media container created with ID: {container_id}")

            # Wait before publishing
            print(f"Waiting {initial_wait} seconds before publishing...")
            time.sleep(initial_wait)

            # Step 2: Publish the thread
            publish_result = publish_thread(access_token, user_id, container_id)
            if publish_result is None or 'id' not in publish_result:
                print("Failed to publish thread.")
                return False

            print(f"Successfully posted to Threads with ID: {publish_result['id']}")
            return True

        except Exception as e:
            print(f"Error posting to Threads: {e}")
            
            if attempt < max_retries - 1:
                print(f"Retrying in {initial_wait} seconds...")
                time.sleep(initial_wait)
            else:
                print("Max retries reached. Failed to post to Threads.")
                return False

    return False

def main():
    category = ["cs.AI", "cs.LG", "cs.CL", "cs.CV", "stat.ML"]
    max_results = 1

    try:
        papers = fetch_latest_papers(category, max_results=max_results)
        if not papers:
            print(f"No recent papers found in category: {category}")
            return

        for paper in papers:
            display(Markdown(f"## Processing: {paper.title}"))
            
            short_id = hashlib.md5(paper.title.encode()).hexdigest()[:8]
            
            date = datetime.now().strftime("%Y-%m-%d")
            if check_existing_post(short_id, date):
                print(f"Blog post already exists for: {paper.title}")
                print("Skipping to next paper...")
                continue

            blog_post = generate_blog_post(paper)
            if not blog_post:
                print(f"Failed to generate blog post for: {paper.title}")
                print("Skipping to next paper...")
                continue

            display(Markdown(f"### Original Paper: [{paper.entry_id}]({paper.entry_id})"))
            display(Markdown(blog_post))
            
            # Generate a temporary post URL
            temp_post_url = f"https://{GITHUB_PAGES_SITE}/{date.replace('-', '/')}/{short_id}/"
            
            # Generate Threads post first
            threads_post = generate_threads_post(paper, temp_post_url)
            if not threads_post:
                print("Failed to generate Threads post. Skipping to next paper...")
                continue
            
            # Generate image based on paper content (fixed: was passing threads_post)
            image_url = generate_ai_image(paper, threads_post)
            if not image_url:
                print("Failed to generate AI image. Skipping to next paper...")
                continue
            
            # Download and save the image to prevent expiration
            image_url = download_and_save_image(image_url, short_id, date)
            
            # Create GitHub blog post with the generated image
            success, post_url = create_github_blog_post(paper, blog_post, date, short_id, image_url)
            
            if success:
                print(f"Successfully created blog post on GitHub: {post_url}")
                
                # Add delay to allow for link preview generation (wait 30 seconds)
                print("Waiting 30 seconds for link preview generation...")
                time.sleep(30)
                
                # Update the Threads post with the correct URL if it changed
                if post_url != temp_post_url:
                    threads_post = threads_post.replace(temp_post_url, post_url)
                
                display(Markdown(f"### Threads Post:\n{threads_post}"))
                if post_to_threads(threads_post, image_url, THREADS_ACCESS_TOKEN, THREADS_USER_ID):
                    print("Successfully posted to Threads with image!")
                else:
                    print("Failed to post to Threads.")
            else:
                print("Failed to create blog post on GitHub.")

    except Exception as e:
        print(f"An error occurred: {e}")
        raise

if __name__ == "__main__":
    main()